# Research Question 3: Market Value

## Reading in Certificate of Occupancy data

In [207]:
import pandas as pd
import numpy as np
import os

In [208]:
co = pd.read_csv(r'/Users/andygoldstein/Documents/OMB Data Work/Working Directory/DOB_Certificate_Of_Occupancy.csv', low_memory=False)

### Examining CO Data

In [209]:
co.shape

(131268, 25)

In [100]:
co.head(2)

,JOB_NUMBER,JOB_TYPE,C_O_ISSUE_DATE,BIN_NUMBER,BOROUGH,NUMBER,STREET,BLOCK,LOT,POSTCODE,...,ISSUE_TYPE,LATITUDE,LONGITUDE,COMMUNITY_BOARD,COUNCIL_DISTRICT,CENSUS_TRACT,BIN,BBL,NTA,LOCATION
0,421557648,A1,10/30/2019,4031824,Queens,43-21,68 STREET,1348,7,11377.0,...,Final,40.741492,-73.896665,2.0,26.0,483.0,4031824.0,4.013480e+09,Elmhurst-Maspeth,"(40.741492, -73.896665)"
1,321174803,A1,10/30/2019,3095121,Brooklyn,542,CRESCENT STREET,4234,34,11208.0,...,Final,40.674745,-73.869849,5.0,42.0,1196.0,3095121.0,3.042340e+09,East New York,"(40.674745, -73.869849)"


In [101]:
co.tail(2)

,JOB_NUMBER,JOB_TYPE,C_O_ISSUE_DATE,BIN_NUMBER,BOROUGH,NUMBER,STREET,BLOCK,LOT,POSTCODE,...,ISSUE_TYPE,LATITUDE,LONGITUDE,COMMUNITY_BOARD,COUNCIL_DISTRICT,CENSUS_TRACT,BIN,BBL,NTA,LOCATION
131266,421559227,A1,07/13/2021,4208885,Queens,89-14,163 ST,9761,55,11432.0,...,Final,40.706659,-73.798047,12.0,24.0,44601.0,4208885.0,4.097610e+09,Jamaica,"(40.706659, -73.798047)"
131267,321154763,NB,07/13/2021,3425539,Brooklyn,892,METROPOLITAN AVE,2916,36,11211.0,...,Final,40.714619,-73.939478,1.0,34.0,481.0,3425539.0,3.029160e+09,East Williamsburg,"(40.714619, -73.939478)"


In [102]:
co.columns

Index(['JOB_NUMBER', 'JOB_TYPE', 'C_O_ISSUE_DATE', 'BIN_NUMBER', 'BOROUGH',
       'NUMBER', 'STREET', 'BLOCK', 'LOT', 'POSTCODE', 'PR_DWELLING_UNIT',
       'EX_DWELLING_UNIT', 'APPLICATION_STATUS_RAW', 'FILING_STATUS_RAW',
       'ITEM_NUMBER', 'ISSUE_TYPE', 'LATITUDE', 'LONGITUDE', 'COMMUNITY_BOARD',
       'COUNCIL_DISTRICT', 'CENSUS_TRACT', 'BIN', 'BBL', 'NTA', 'LOCATION'],
      dtype='object')

### Subsetting by Job Type & Date

In [104]:
# subsetting by job type
co = co.loc[co['JOB_TYPE'] == 'NB']

In [105]:
# converting issue date to datetime
co['C_O_ISSUE_DATE']= pd.to_datetime(co['C_O_ISSUE_DATE'])

In [106]:
print(co['C_O_ISSUE_DATE'].dtype)

datetime64[ns]


In [107]:
co = co.groupby(["JOB_NUMBER"], group_keys=False).apply(lambda g: g.nsmallest(1, "C_O_ISSUE_DATE"))

In [108]:
# check for duplicates
dup = co.duplicated(['BBL'], keep=False)

co['DUPLICATE'] = np.select([dup],['yes'], default='no')
print (co)

        JOB_NUMBER JOB_TYPE C_O_ISSUE_DATE  BIN_NUMBER        BOROUGH  NUMBER  \
26974    100075133       NB     2012-07-30     1035742      Manhattan     693   
17481    100089583       NB     2020-02-19     1804283      Manhattan    2006   
27181    100129790       NB     2012-09-19     1087519      Manhattan     314   
28089    100137219       NB     2012-08-16     1084667      Manhattan     745   
27209    100363562       NB     2012-08-08     1084564      Manhattan    1992   
...            ...      ...            ...         ...            ...     ...   
130348   520397106       NB     2021-04-12     5174633  Staten Island     638   
126867   520451840       NB     2020-12-28     5174691  Staten Island  123GAR   
131056   520452215       NB     2021-06-22     5174896  Staten Island      34   
128995   520454339       NB     2021-02-12     5174900  Staten Island     347   
129045   520454348       NB     2021-02-16     5174685  Staten Island     343   

                    STREET 

In [109]:
# duplicate check Pre-date filter
pd.crosstab(index=co['DUPLICATE'], columns='BBL')

col_0,BBL
DUPLICATE,
no,11918
yes,5668


In [110]:
# subsetting by Certifice of Occupancy date
co = co.loc[co['C_O_ISSUE_DATE'] >= '7/1/2016']

In [111]:
# subsetting by Certifice of Occupancy date
co = co.loc[co['C_O_ISSUE_DATE'] <= '6/30/2020']
co

,JOB_NUMBER,JOB_TYPE,C_O_ISSUE_DATE,BIN_NUMBER,BOROUGH,NUMBER,STREET,BLOCK,LOT,POSTCODE,...,LATITUDE,LONGITUDE,COMMUNITY_BOARD,COUNCIL_DISTRICT,CENSUS_TRACT,BIN,BBL,NTA,LOCATION,DUPLICATE
17481,100089583,NB,2020-02-19,1804283,Manhattan,2006,LEXINGTON AVENUE,1771,17,10035.0,...,40.802543,-73.938773,11.0,9.0,196.0,1000000.0,1.017710e+09,East Harlem North,"(40.802543, -73.938773)",no
122315,100771513,NB,2019-01-08,1053496,Manhattan,75,WEST 125 STREET,1723,5,10027.0,...,40.807440,-73.944674,10.0,9.0,200.0,1053496.0,1.017230e+09,Central Harlem South,"(40.80744, -73.944674)",no
113236,102431904,NB,2018-06-26,1087556,Manhattan,450,WEST BROADWAY,516,37,10012.0,...,40.726053,-74.000675,2.0,1.0,49.0,1087556.0,1.005160e+09,SoHo-TriBeCa-Civic Center-Little Italy,"(40.726053, -74.000675)",no
121026,103042752,NB,2018-12-24,1008438,Manhattan,57,BOND STREET,529,38,10012.0,...,40.725874,-73.992568,2.0,1.0,5502.0,1008438.0,1.005298e+09,West Village,"(40.725874, -73.992568)",no
30740,103131086,NB,2016-08-01,1087891,Manhattan,30,EAST 129 STREET,1753,58,10035.0,...,40.808458,-73.939237,11.0,9.0,206.0,1087891.0,1.017530e+09,Central Harlem North-Polo Grounds,"(40.808458, -73.939237)",no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14329,520378190,NB,2020-03-27,5171753,Staten Island,113 GAR,CANTON AVENUE,6013,46,10312.0,...,40.548747,-74.191785,3.0,51.0,17010.0,NaN,NaN,Arden Heights,"(40.548747, -74.191785)",yes
10001,520378207,NB,2020-05-21,5171817,Staten Island,117,CANTON AVENUE,6013,47,10312.0,...,40.548785,-74.191857,3.0,51.0,17010.0,NaN,NaN,Arden Heights,"(40.548785, -74.191857)",yes
14330,520378216,NB,2020-03-27,5171818,Staten Island,117 GAR,CANTON AVENUE,6013,47,10312.0,...,40.548785,-74.191857,3.0,51.0,17010.0,NaN,NaN,Arden Heights,"(40.548785, -74.191857)",yes
10003,520384058,NB,2020-05-21,5123917,Staten Island,52,BENNETT AVE,6249,627,10312.0,...,40.538021,-74.180839,3.0,51.0,17005.0,5123917.0,5.062491e+09,Annadale-Huguenot-Prince's Bay-Eltingville,"(40.538021, -74.180839)",no


In [112]:
# duplicate check Post-date filter
pd.crosstab(index=co['DUPLICATE'], columns='BBL')

col_0,BBL
DUPLICATE,
no,4543
yes,3346


### Creating PARID and Deduplicating

In [113]:
# creating Boro #s
# Key Program Type column
BORO = []
for row in co['BOROUGH']:
    if row == 'Manhattan': BORO.append('1')
    elif row == 'Bronx': BORO.append('2')
    elif row == 'Brooklyn': BORO.append('3')
    elif row == 'Queens': BORO.append('4')
    elif row == 'Staten Island': BORO.append('5')
        
    else: keyprogram.append('')

In [114]:
co['BORO'] = BORO
co['BORO']

17481     1
122315    1
113236    1
121026    1
30740     1
         ..
14329     5
10001     5
14330     5
10003     5
17731     5
Name: BORO, Length: 7889, dtype: object

In [115]:
co['BLOCK']

17481     1771
122315    1723
113236     516
121026     529
30740     1753
          ... 
14329     6013
10001     6013
14330     6013
10003     6249
17731     4245
Name: BLOCK, Length: 7889, dtype: int64

In [116]:
co['BLOCK']= pd.Series(co['BLOCK']).astype(str).str.zfill(5)
co['BLOCK']

17481     01771
122315    01723
113236    00516
121026    00529
30740     01753
          ...  
14329     06013
10001     06013
14330     06013
10003     06249
17731     04245
Name: BLOCK, Length: 7889, dtype: object

In [117]:
co['LOT']

17481      17
122315      5
113236     37
121026     38
30740      58
         ... 
14329      46
10001      47
14330      47
10003     627
17731      68
Name: LOT, Length: 7889, dtype: object

In [118]:
# normalizing LOT character length for concat purposes
co['LOT']= pd.Series(co['LOT']).astype(str).str.zfill(4)
co['LOT']

17481     0017
122315    0005
113236    0037
121026    0038
30740     0058
          ... 
14329     0046
10001     0047
14330     0047
10003     0627
17731     0068
Name: LOT, Length: 7889, dtype: object

In [119]:
# creating PARID as unique identifier for BBL
co['PARID'] = co['BORO'] + co['BLOCK'] + co['LOT']
co['PARID']

17481     1017710017
122315    1017230005
113236    1005160037
121026    1005290038
30740     1017530058
             ...    
14329     5060130046
10001     5060130047
14330     5060130047
10003     5062490627
17731     5042450068
Name: PARID, Length: 7889, dtype: object

In [120]:
paridduplicate = co.duplicated(['PARID'], keep=False)

co['PARIDDUPLICATE'] = np.select([paridduplicate],['yes'], default='no')
print (co)

        JOB_NUMBER JOB_TYPE C_O_ISSUE_DATE  BIN_NUMBER        BOROUGH  \
17481    100089583       NB     2020-02-19     1804283      Manhattan   
122315   100771513       NB     2019-01-08     1053496      Manhattan   
113236   102431904       NB     2018-06-26     1087556      Manhattan   
121026   103042752       NB     2018-12-24     1008438      Manhattan   
30740    103131086       NB     2016-08-01     1087891      Manhattan   
...            ...      ...            ...         ...            ...   
14329    520378190       NB     2020-03-27     5171753  Staten Island   
10001    520378207       NB     2020-05-21     5171817  Staten Island   
14330    520378216       NB     2020-03-27     5171818  Staten Island   
10003    520384058       NB     2020-05-21     5123917  Staten Island   
17731    520386430       NB     2020-02-21     5171779  Staten Island   

         NUMBER            STREET  BLOCK   LOT  POSTCODE  ...  \
17481      2006  LEXINGTON AVENUE  01771  0017   10035.0  

In [121]:
# PARID duplicate check
pd.crosstab(index=co['PARIDDUPLICATE'], columns='PARID')

col_0,PARID
PARIDDUPLICATE,
no,6231
yes,1658


In [122]:
# group by latest
co=co.groupby(["PARID"], group_keys=False).apply(lambda g: g.nlargest(1, "C_O_ISSUE_DATE"))

In [123]:
# duplicate check with latest
pd.crosstab(index=co['PARIDDUPLICATE'], columns='PARID')

col_0,PARID
PARIDDUPLICATE,
no,6231
yes,676


## Reading in Property Assessment data

In [135]:
# filtered for tax class 2 & market value =/= 0
rpad2 = pd.read_csv('/Users/andygoldstein/Documents/OMB Data Work/Working Directory/Property_Valuation_and_Assessment_Data_Tax_Classes_1_2_3_4.csv')
rpad2

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,4,22,33,69,74,75,77,78,87,92,99,115,117) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,PARID,BORO,BLOCK,LOT,EASEMENT,SUBIDENT-REUC,RECTYPE,YEAR,IDENT,SUBIDENT,...,STORAGE_AREA_GROSS,GARAGE_AREA,OTHER_AREA_GROSS,REUC_DESCRIPTION,EXTRACRDT,PYTAXFLAG,TENTAXFLAG,CBNTAXFLAG,FINTAXFLAG,CURTAXFLAG
0,1000081002,1,8,1002,NaN,NaN,1,2021,NaN,NaN,...,0,0,0,NaN,05/17/2020,T,T,T,T,T
1,1000110014,1,11,14,NaN,NaN,1,2021,NaN,NaN,...,0,0,0,NaN,05/17/2020,T,T,T,T,T
2,1000160015,1,16,15,NaN,NaN,1,2021,NaN,NaN,...,0,8400,0,NaN,05/17/2020,A,A,A,A,A
3,1000160020,1,16,20,NaN,NaN,1,2021,NaN,NaN,...,0,1,0,NaN,05/17/2020,A,A,A,A,A
4,1000160100,1,16,100,NaN,NaN,1,2021,NaN,NaN,...,0,48112,0,NaN,05/17/2020,A,A,A,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269585,5054911152,5,5491,1152,NaN,NaN,1,2022,NaN,NaN,...,0,0,0,NaN,05/24/2021,T,A,A,A,A
269586,5054911154,5,5491,1154,NaN,NaN,1,2022,NaN,NaN,...,0,0,0,NaN,05/24/2021,T,A,A,A,A
269587,5054911155,5,5491,1155,NaN,NaN,1,2022,NaN,NaN,...,0,0,0,NaN,05/24/2021,T,A,A,A,A
269588,5054911162,5,5491,1162,NaN,NaN,1,2022,NaN,NaN,...,0,0,0,NaN,05/24/2021,T,A,A,A,A


### Examing RPAD data

In [136]:
rpad2.shape

(269590, 139)

In [137]:
rpad2.head(2)

,PARID,BORO,BLOCK,LOT,EASEMENT,SUBIDENT-REUC,RECTYPE,YEAR,IDENT,SUBIDENT,...,STORAGE_AREA_GROSS,GARAGE_AREA,OTHER_AREA_GROSS,REUC_DESCRIPTION,EXTRACRDT,PYTAXFLAG,TENTAXFLAG,CBNTAXFLAG,FINTAXFLAG,CURTAXFLAG
0,1000081002,1,8,1002,NaN,NaN,1,2021,NaN,NaN,...,0,0,0,NaN,05/17/2020,T,T,T,T,T
1,1000110014,1,11,14,NaN,NaN,1,2021,NaN,NaN,...,0,0,0,NaN,05/17/2020,T,T,T,T,T


In [138]:
rpad2.tail(2)

,PARID,BORO,BLOCK,LOT,EASEMENT,SUBIDENT-REUC,RECTYPE,YEAR,IDENT,SUBIDENT,...,STORAGE_AREA_GROSS,GARAGE_AREA,OTHER_AREA_GROSS,REUC_DESCRIPTION,EXTRACRDT,PYTAXFLAG,TENTAXFLAG,CBNTAXFLAG,FINTAXFLAG,CURTAXFLAG
269588,5054911162,5,5491,1162,NaN,NaN,1,2022,NaN,NaN,...,0,0,0,NaN,05/24/2021,T,A,A,A,A
269589,5054911164,5,5491,1164,NaN,NaN,1,2022,NaN,NaN,...,0,0,0,NaN,05/24/2021,T,A,A,A,A


In [139]:
for col in rpad2.columns:
    print(col)

PARID
BORO
BLOCK
LOT
EASEMENT
SUBIDENT-REUC
RECTYPE
YEAR
IDENT
SUBIDENT
ROLL_SECTION
SECVOL
PYMKTLAND
PYMKTTOT
PYACTLAND
PYACTTOT
PYACTEXTOT
PYTRNLAND
PYTRNTOT
PYTRNEXTOT
PYTXBTOT
PYTXBEXTOT
PYTAXCLASS
TENMKTLAND
TENMKTTOT
TENACTLAND
TENACTTOT
TENACTEXTOT
TENTRNLAND
TENTRNTOT
TENTRNEXTOT
TENTXBTOT
TENTXBEXTOT
TENTAXCLASS
CBNMKTLAND
CBNMKTTOT
CBNACTLAND
CBNACTTOT
CBNACTEXTOT
CBNTRNLAND
CBNTRNTOT
CBNTRNEXTOT
CBNTXBTOT
CBNTXBEXTOT
CBNTAXCLASS
FINMKTLAND
FINMKTTOT
FINACTLAND
FINACTTOT
FINACTEXTOT
FINTRNLAND
FINTRNTOT
FINTRNEXTOT
FINTXBTOT
FINTXBEXTOT
FINTAXCLASS
CURMKTLAND
CURMKTTOT
CURACTLAND
CURACTTOT
CURACTEXTOT
CURTRNLAND
CURTRNTOT
CURTRNEXTOT
CURTXBTOT
CURTXBEXTOT
CURTAXCLASS
PERIOD
NEWDROP
NOAV
VALREF
BLDG_CLASS
OWNER
ZONING
HOUSENUM_LO
HOUSENUM_HI
STREET_NAME
ZIP_CODE
GEPSUPPORT_RC
STCODE
LOT_FRT
LOT_DEP
LOT_IRREG
BLD_FRT
BLD_DEP
BLD_EXT
BLD_STORY
CORNER
LAND_AREA
NUM_BLDGS
YRBUILT
YRBUILT_RANGE
YRBUILT_FLAG
YRALT1
YRALT1_RANGE
YRALT2
YRALT2_RANGE
COOP_APTS
UNITS
REUC_REF
APTNO
COOP

In [140]:
rpad.describe()

,BORO,BLOCK,LOT,SUBIDENT-REUC,RECTYPE,YEAR,IDENT,SUBIDENT,ROLL_SECTION,SECVOL,...,OFFICE_AREA_GROSS,RESIDENTIAL_AREA_GROSS,RETAIL_AREA_GROSS,LOFT_AREA_GROSS,FACTORY_AREA_GROSS,WAREHOUSE_AREA_GROSS,STORAGE_AREA_GROSS,GARAGE_AREA,OTHER_AREA_GROSS,REUC_DESCRIPTION
count,269590.000000,269590.000000,269590.000000,0.0,269590.0,269590.000000,0.0,0.0,0.0,269590.000000,...,269590.000000,2.695900e+05,269590.000000,269590.0,269590.000000,269590.000000,269590.000000,269590.000000,269590.000000,0.0
mean,2.368000,2690.856564,1491.852187,NaN,1.0,2020.979328,NaN,NaN,NaN,1037.553162,...,48.691428,1.159076e+04,135.573575,0.0,0.337720,0.159973,6.805579,155.303468,54.844189,NaN
std,1.149643,2528.688425,1227.890472,NaN,0.0,0.816832,NaN,NaN,NaN,954.018748,...,971.571024,7.831681e+04,2444.501380,0.0,84.616659,32.119251,339.894325,4595.263757,2425.734433,NaN
min,1.000000,1.000000,1.000000,NaN,1.0,2020.000000,NaN,NaN,NaN,101.000000,...,0.000000,0.000000e+00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,1.000000,992.000000,1019.000000,NaN,1.0,2020.000000,NaN,NaN,NaN,403.000000,...,0.000000,5.910000e+02,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
50%,2.000000,1986.000000,1124.000000,NaN,1.0,2021.000000,NaN,NaN,NaN,709.000000,...,0.000000,8.290000e+02,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
75%,3.000000,3943.000000,1495.000000,NaN,1.0,2022.000000,NaN,NaN,NaN,1503.000000,...,0.000000,1.303000e+03,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
max,5.000000,16250.000000,9978.000000,NaN,1.0,2022.000000,NaN,NaN,NaN,6302.000000,...,100000.000000,1.321140e+07,537154.000000,0.0,24764.000000,11681.000000,67500.000000,945000.000000,476973.000000,NaN


In [141]:
# checking tax class filter
rpad2['CURTAXCLASS']

0         2
1         2
2         2
3         2
4         2
         ..
269585    2
269586    2
269587    2
269588    2
269589    2
Name: CURTAXCLASS, Length: 269590, dtype: int64

In [144]:
# dropping repeated columns before merge
rpad2 = rpad2.drop(['BLOCK'], axis=1)

In [145]:
rpad2 = rpad2.drop(['LOT'], axis=1)

## Merging CO & RPAD data

In [154]:
corpad2 = co.merge(rpad2, how = 'left', on = 'PARID')
corpad2

,JOB_NUMBER,JOB_TYPE,C_O_ISSUE_DATE,BIN_NUMBER,BOROUGH,NUMBER,STREET,BLOCK,LOT,POSTCODE,...,STORAGE_AREA_GROSS,GARAGE_AREA,OTHER_AREA_GROSS,REUC_DESCRIPTION,EXTRACRDT,PYTAXFLAG,TENTAXFLAG,CBNTAXFLAG,FINTAXFLAG,CURTAXFLAG
0,199990002,NB,2016-10-02,1813360,Manhattan,1,BOGUS LANE,00001,0001,10001.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,121330764,NB,2017-09-19,1000017,Manhattan,6,WATER STREET,00008,0051,10004.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,110017151,NB,2016-12-30,1090192,Manhattan,50,WEST STREET,00017,7501,10006.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,121186885,NB,2019-11-15,1090830,Manhattan,68,TRINITY PLACE,00051,0007,10006.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,121324147,NB,2016-11-28,1089248,Manhattan,133,GREENWICH STREET,00052,0008,10006.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6956,520234737,NB,2016-07-18,5089051,Staten Island,90,CRAIG AVENUE,08039,0034,10307.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6957,520234746,NB,2016-07-15,5089052,Staten Island,88,CRAIG AVENUE,08039,0037,10307.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6958,520180107,NB,2020-02-19,5158963,Staten Island,203,JOHNSON AVENUE,08046,0144,10307.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6959,520321394,NB,2019-06-28,5170817,Staten Island,141,BENTLEY STREET,08049,0030,10307.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [156]:
for col in corpad2.columns:
    print(col)

JOB_NUMBER
JOB_TYPE
C_O_ISSUE_DATE
BIN_NUMBER
BOROUGH
NUMBER
STREET
BLOCK
LOT
POSTCODE
PR_DWELLING_UNIT
EX_DWELLING_UNIT
APPLICATION_STATUS_RAW
FILING_STATUS_RAW
ITEM_NUMBER
ISSUE_TYPE
LATITUDE
LONGITUDE
COMMUNITY_BOARD
COUNCIL_DISTRICT
CENSUS_TRACT
BIN
BBL
NTA
LOCATION
DUPLICATE
BORO_x
PARID
PARIDDUPLICATE
BORO_y
EASEMENT
SUBIDENT-REUC
RECTYPE
YEAR
IDENT
SUBIDENT
ROLL_SECTION
SECVOL
PYMKTLAND
PYMKTTOT
PYACTLAND
PYACTTOT
PYACTEXTOT
PYTRNLAND
PYTRNTOT
PYTRNEXTOT
PYTXBTOT
PYTXBEXTOT
PYTAXCLASS
TENMKTLAND
TENMKTTOT
TENACTLAND
TENACTTOT
TENACTEXTOT
TENTRNLAND
TENTRNTOT
TENTRNEXTOT
TENTXBTOT
TENTXBEXTOT
TENTAXCLASS
CBNMKTLAND
CBNMKTTOT
CBNACTLAND
CBNACTTOT
CBNACTEXTOT
CBNTRNLAND
CBNTRNTOT
CBNTRNEXTOT
CBNTXBTOT
CBNTXBEXTOT
CBNTAXCLASS
FINMKTLAND
FINMKTTOT
FINACTLAND
FINACTTOT
FINACTEXTOT
FINTRNLAND
FINTRNTOT
FINTRNEXTOT
FINTXBTOT
FINTXBEXTOT
FINTAXCLASS
CURMKTLAND
CURMKTTOT
CURACTLAND
CURACTTOT
CURACTEXTOT
CURTRNLAND
CURTRNTOT
CURTRNEXTOT
CURTXBTOT
CURTXBEXTOT
CURTAXCLASS
PERIOD
NEWDROP
NOAV

## Borough & Community District Sample Size

### Bronx Sample Size

In [158]:
# Bronx subset sample size
bx = corpad2[(corpad2.BOROUGH =='Bronx')]
bx

,JOB_NUMBER,JOB_TYPE,C_O_ISSUE_DATE,BIN_NUMBER,BOROUGH,NUMBER,STREET,BLOCK,LOT,POSTCODE,...,STORAGE_AREA_GROSS,GARAGE_AREA,OTHER_AREA_GROSS,REUC_DESCRIPTION,EXTRACRDT,PYTAXFLAG,TENTAXFLAG,CBNTAXFLAG,FINTAXFLAG,CURTAXFLAG
360,220407842,NB,2018-08-16,2128617,Bronx,532,E 142 STREET,02268,0018,10454.0,...,0.0,0.0,0.0,NaN,05/17/2020,A,A,A,A,A
361,220407842,NB,2018-08-16,2128617,Bronx,532,E 142 STREET,02268,0018,10454.0,...,0.0,0.0,0.0,NaN,05/24/2021,A,A,A,A,A
362,200483039,NB,2017-11-29,2117110,Bronx,521,EAST 149 STREET,02276,0054,10455.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
363,220516216,NB,2020-05-29,2127605,Bronx,485,EAST 139TH STREET,02284,0034,10454.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364,220471683,NB,2018-03-22,2818714,Bronx,352,WILLIS AVE,02286,0012,10454.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012,220351722,NB,2020-06-17,2117817,Bronx,680-14,W 246 STREET,05920,0398,10471.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1013,220422923,NB,2016-08-26,2090709,Bronx,4900,PALISADES AVENUE,05937,00360,10471.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1014,201093725,NB,2020-01-31,2129285,Bronx,620,WEST 256 STREET,05949,0294,10471.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1015,201098793,NB,2020-01-10,2129286,Bronx,622,WEST 256 STREET,05949,0295,10471.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
# Bronx by CD sample size
bx['COMMUNITY_BOARD'].value_counts()

12.0    111
10.0     73
9.0      67
3.0      62
1.0      57
11.0     56
4.0      52
6.0      47
7.0      35
8.0      31
5.0      30
2.0      29
Name: COMMUNITY_BOARD, dtype: int64

### Brooklyn Sample Size

In [160]:
# Brooklyn subset sample size
bk = corpad2[(corpad2.BOROUGH =='Brooklyn')]
bk

,JOB_NUMBER,JOB_TYPE,C_O_ISSUE_DATE,BIN_NUMBER,BOROUGH,NUMBER,STREET,BLOCK,LOT,POSTCODE,...,STORAGE_AREA_GROSS,GARAGE_AREA,OTHER_AREA_GROSS,REUC_DESCRIPTION,EXTRACRDT,PYTAXFLAG,TENTAXFLAG,CBNTAXFLAG,FINTAXFLAG,CURTAXFLAG
1017,320594727,NB,2016-07-06,3413929,Brooklyn,1,JOHN ST,00001,00002,11201.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1018,320590525,NB,2019-12-11,3425340,Brooklyn,9,OLD FULTON STREET,00035,0010,11201.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1019,320877627,NB,2018-07-06,3413894,Brooklyn,181,FRONT STREET,00041,0042,11201.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1020,320914445,NB,2018-08-23,3426023,Brooklyn,120,NASSAU STREET,00107,0009,11201.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1021,301908933,NB,2018-06-20,3000189,Brooklyn,187,BRIDGE STREET,00108,0009,11201.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3217,310053919,NB,2019-11-26,3424370,Brooklyn,2738,GERRITSEN AVENUE,08925,0174,11229.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3218,310053900,NB,2019-11-27,3424518,Brooklyn,2736,GERRITSEN AVENUE,08925,0175,11229.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3219,321183697,NB,2018-02-23,3249838,Brooklyn,7,BEVY COURT,08925,0266,11229.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3220,321185864,NB,2018-03-29,3249944,Brooklyn,28,ASTER COURT,08932,0544,11229.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [161]:
# Brooklyn by CD sample size
bk['COMMUNITY_BOARD'].value_counts()

1.0     308
3.0     269
4.0     176
12.0    165
15.0    151
5.0     145
2.0     125
6.0     121
8.0      96
13.0     76
9.0      71
17.0     68
14.0     60
11.0     59
18.0     58
16.0     53
7.0      45
10.0     31
Name: COMMUNITY_BOARD, dtype: int64

### Queens Sample Size

In [162]:
# Queens subset sample size
qn = corpad2[(corpad2.BOROUGH =='Queens')]
qn

,JOB_NUMBER,JOB_TYPE,C_O_ISSUE_DATE,BIN_NUMBER,BOROUGH,NUMBER,STREET,BLOCK,LOT,POSTCODE,...,STORAGE_AREA_GROSS,GARAGE_AREA,OTHER_AREA_GROSS,REUC_DESCRIPTION,EXTRACRDT,PYTAXFLAG,TENTAXFLAG,CBNTAXFLAG,FINTAXFLAG,CURTAXFLAG
3222,421400638,NB,2019-12-13,4615610,Queens,2-22,51ST AVENUE,00015,0034,11101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3223,420651440,NB,2019-09-20,4594828,Queens,4-44,47 ROAD,00019,0021,11101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3224,421209169,NB,2019-07-18,4618965,Queens,46-02,VERNON BOULEVARD,00027,0046,11101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3225,410114326,NB,2017-02-13,4541432,Queens,47-05,5 STREET,00029,00026,11101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3226,420559540,NB,2019-05-20,4618263,Queens,5-26,47TH AVENUE,00029,0034,11101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5094,421642066,NB,2020-06-16,4529957,Queens,282,SEABREEZE AVE,16340,0050,11697.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5095,420656551,NB,2017-06-30,4464905,Queens,851,LIBERTY LANE,16350,00300,11697.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5096,420948390,NB,2017-06-22,4466529,Queens,141,OCEANSIDE AVE,16350,00400,11697.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5097,420858405,NB,2020-06-05,4464543,Queens,2,BAYSIDE DRIVE,16350,0300,11697.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
# Queens by CD sample size
qn['COMMUNITY_BOARD'].value_counts()

7.0     372
14.0    266
11.0    206
12.0    179
1.0     166
4.0     138
13.0     98
2.0      93
10.0     88
8.0      73
5.0      59
3.0      57
9.0      36
6.0      34
81.0      2
Name: COMMUNITY_BOARD, dtype: int64

### Staten Island Sample Size

In [164]:
# Staten Island subset sample size
si = corpad2[(corpad2.BOROUGH =='Staten Island')]
si

,JOB_NUMBER,JOB_TYPE,C_O_ISSUE_DATE,BIN_NUMBER,BOROUGH,NUMBER,STREET,BLOCK,LOT,POSTCODE,...,STORAGE_AREA_GROSS,GARAGE_AREA,OTHER_AREA_GROSS,REUC_DESCRIPTION,EXTRACRDT,PYTAXFLAG,TENTAXFLAG,CBNTAXFLAG,FINTAXFLAG,CURTAXFLAG
5099,520208800,NB,2019-12-09,5164580,Staten Island,35,RICHMOND TERRACE,00002,0015,10301.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5100,500871602,NB,2017-08-28,5165063,Staten Island,105B,HAMILTON AVENUE,00022,0001,10301.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5101,520102335,NB,2016-08-17,5158649,Staten Island,255,WESTERVELT AVENUE,00028,00025,10301.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5102,520253993,NB,2016-08-24,5804879,Staten Island,192,CORSON AVENUE,00035,0022,10301.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5103,520264384,NB,2017-03-30,5166617,Staten Island,126,SCRIBNER AVENUE,00036,0018,10301.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6956,520234737,NB,2016-07-18,5089051,Staten Island,90,CRAIG AVENUE,08039,0034,10307.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6957,520234746,NB,2016-07-15,5089052,Staten Island,88,CRAIG AVENUE,08039,0037,10307.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6958,520180107,NB,2020-02-19,5158963,Staten Island,203,JOHNSON AVENUE,08046,0144,10307.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6959,520321394,NB,2019-06-28,5170817,Staten Island,141,BENTLEY STREET,08049,0030,10307.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [165]:
# Staten Island by CD sample size
si['COMMUNITY_BOARD'].value_counts()

2.0    627
3.0    572
1.0    492
Name: COMMUNITY_BOARD, dtype: int64

## Case Study CD Subsets & Variables of Interest

### Bronx - CD 04

In [166]:
# Bronx-04 subset
bx4 = corpad2[(corpad2.BOROUGH =='Bronx') & (corpad2.COMMUNITY_BOARD == 4)]
bx4

,JOB_NUMBER,JOB_TYPE,C_O_ISSUE_DATE,BIN_NUMBER,BOROUGH,NUMBER,STREET,BLOCK,LOT,POSTCODE,...,STORAGE_AREA_GROSS,GARAGE_AREA,OTHER_AREA_GROSS,REUC_DESCRIPTION,EXTRACRDT,PYTAXFLAG,TENTAXFLAG,CBNTAXFLAG,FINTAXFLAG,CURTAXFLAG
391,220493231,NB,2019-05-07,2128704,Bronx,646,GERARD AVENUE,02353,0020,10451.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
413,220354952,NB,2016-12-13,2001935,Bronx,316,E. 162 STREET,02421,00027,10451.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
414,220151911,NB,2018-04-04,2001931,Bronx,294,EAST 162ND STREET,02421,0018,10451.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
415,220579817,NB,2018-08-30,2128408,Bronx,381,E 166 STREET,02426,0001,10456.0,...,0.0,0.0,0.0,NaN,05/17/2020,T,A,A,A,A
416,220579817,NB,2018-08-30,2128408,Bronx,381,E 166 STREET,02426,0001,10456.0,...,0.0,0.0,0.0,NaN,05/24/2021,A,A,A,A,A
417,220619150,NB,2019-04-30,2129110,Bronx,316,EAST 165 STREET,02432,0080,10456.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
418,220456432,NB,2018-06-18,2002238,Bronx,1067,TELLER AVENUE,02433,0067,10456.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
419,220421390,NB,2018-11-07,2129116,Bronx,288,EAST 169 STREET,02439,0053,10456.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
422,201061288,NB,2020-06-11,2124760,Bronx,1120,SHERMAN AVENUE,02452,0008,10456.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423,220579657,NB,2019-07-24,2098340,Bronx,1181,SHERMAN AVE,02456,0240,10456.0,...,0.0,0.0,0.0,NaN,05/24/2021,A,A,A,A,A


In [167]:
# variables of interest
pd.pivot_table(bx4, values = ['FINMKTLAND','FINMKTTOT','FINACTLAND','FINACTTOT','FINACTEXTOT','FINTRNLAND','FINTRNTOT','FINTRNEXTOT','FINTXBTOT','FINTXBEXTOT'], index = ['COMMUNITY_BOARD'])

,FINACTEXTOT,FINACTLAND,FINACTTOT,FINMKTLAND,FINMKTTOT,FINTRNEXTOT,FINTRNLAND,FINTRNTOT,FINTXBEXTOT,FINTXBTOT
COMMUNITY_BOARD,,,,,,,,,,
4.0,1.984797e+06,202761.290323,2.026742e+06,450580.645161,4.503871e+06,2.072927e+06,181953.064516,2.117466e+06,1.984797e+06,2.026742e+06


### Bronx - CD 09

In [173]:
# Bronx-07 subset
bx7 = corpad2[(corpad2.BOROUGH =='Bronx') & (corpad2.COMMUNITY_BOARD == 7)]
bx7

,JOB_NUMBER,JOB_TYPE,C_O_ISSUE_DATE,BIN_NUMBER,BOROUGH,NUMBER,STREET,BLOCK,LOT,POSTCODE,...,STORAGE_AREA_GROSS,GARAGE_AREA,OTHER_AREA_GROSS,REUC_DESCRIPTION,EXTRACRDT,PYTAXFLAG,TENTAXFLAG,CBNTAXFLAG,FINTAXFLAG,CURTAXFLAG
624,220102092,NB,2018-10-16,2127056,Bronx,2605,GRAND CONCOURSE,03168,7501,10468.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
627,220152616,NB,2020-02-28,2803496,Bronx,2519,CRESTON AVENUE,03175,0026,10468.0,...,0.0,0.0,0.0,NaN,05/17/2020,A,A,A,A,A
628,220152616,NB,2020-02-28,2803496,Bronx,2519,CRESTON AVENUE,03175,0026,10468.0,...,0.0,0.0,0.0,NaN,05/24/2021,A,A,A,A,A
634,220353221,NB,2017-05-25,2124539,Bronx,2536,GRAND AVE,03204,00022,10468.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
635,220353212,NB,2017-05-23,2014433,Bronx,2538,GRAND AVE,03204,00023,10468.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
638,220420202,NB,2017-06-02,2014915,Bronx,2264,LORING PLACE NORTH,03225,00010,10468.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
639,210071624,NB,2017-12-06,2125050,Bronx,2186,CEDAR AVE.,03232,0102,10468.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
640,210071633,NB,2017-12-06,2125038,Bronx,2184,CEDAR AVE.,03232,0103,10468.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
641,220151608,NB,2018-01-25,2811610,Bronx,233,LANDING ROAD,03236,0025,10468.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
642,220353187,NB,2017-10-18,2128681,Bronx,166,WEST KINGSBRIDGE RD,03240,0059,10463.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
# variables of interest
pd.pivot_table(bx7, values = ['FINMKTLAND','FINMKTTOT','FINACTLAND','FINACTTOT','FINACTEXTOT','FINTRNLAND','FINTRNTOT','FINTRNEXTOT','FINTXBTOT','FINTXBEXTOT'], index = ['COMMUNITY_BOARD'])

,FINACTEXTOT,FINACTLAND,FINACTTOT,FINMKTLAND,FINMKTTOT,FINTRNEXTOT,FINTRNLAND,FINTRNTOT,FINTXBEXTOT,FINTXBTOT
COMMUNITY_BOARD,,,,,,,,,,
7.0,2.036816e+06,226387.5,2164365.0,503083.333333,4809700.0,2.110828e+06,208608.416667,2.234757e+06,2.024764e+06,2.146579e+06


### Bronx - CD 05

In [181]:
# Bronx-05 subset
bx5 = corpad2[(corpad2.BOROUGH =='Bronx') & (corpad2.COMMUNITY_BOARD == 5)]
bx5

,JOB_NUMBER,JOB_TYPE,C_O_ISSUE_DATE,BIN_NUMBER,BOROUGH,NUMBER,STREET,BLOCK,LOT,POSTCODE,...,STORAGE_AREA_GROSS,GARAGE_AREA,OTHER_AREA_GROSS,REUC_DESCRIPTION,EXTRACRDT,PYTAXFLAG,TENTAXFLAG,CBNTAXFLAG,FINTAXFLAG,CURTAXFLAG
520,220471718,NB,2017-12-15,2817614,Bronx,1779,WEEKS AVENUE,02796,0036,10457.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
521,200929237,NB,2019-03-28,2129269,Bronx,1752,MONROE AVENUE,02798,0014,10457.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
522,220423085,NB,2018-06-05,2007561,Bronx,250,MT HOPE PLACE,02802,0029,10457.0,...,0.0,0.0,0.0,NaN,05/17/2020,T,A,A,A,A
523,220423085,NB,2018-06-05,2007561,Bronx,250,MT HOPE PLACE,02802,0029,10457.0,...,0.0,0.0,0.0,NaN,05/24/2021,A,A,A,A,A
524,220392858,NB,2019-02-27,2128390,Bronx,2028,CRESTON AVENUE,02808,0073,10453.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
525,220482868,NB,2019-07-18,2128866,Bronx,240,EAST 178 STREET,02810,0024,10457.0,...,0.0,0.0,0.0,NaN,05/17/2020,T,A,A,A,A
526,220482868,NB,2019-07-18,2128866,Bronx,240,EAST 178 STREET,02810,0024,10457.0,...,0.0,0.0,0.0,NaN,05/24/2021,A,A,A,A,A
527,220081186,NB,2017-10-26,2007780,Bronx,280,EAST BURNSIDE AVE,02814,0073,10457.0,...,0.0,0.0,0.0,NaN,05/17/2020,A,A,A,A,A
528,220081186,NB,2017-10-26,2007780,Bronx,280,EAST BURNSIDE AVE,02814,0073,10457.0,...,0.0,0.0,0.0,NaN,05/24/2021,A,A,A,A,A
535,220357423,NB,2017-01-04,2124607,Bronx,74,WEST TREMONT AVENUE,02862,00006,10453.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [182]:
# variables of interest
pd.pivot_table(bx5, values = ['FINMKTLAND','FINMKTTOT','FINACTLAND','FINACTTOT','FINACTEXTOT','FINTRNLAND','FINTRNTOT','FINTRNEXTOT','FINTXBTOT','FINTXBEXTOT'], index = ['COMMUNITY_BOARD'])

,FINACTEXTOT,FINACTLAND,FINACTTOT,FINMKTLAND,FINMKTTOT,FINTRNEXTOT,FINTRNLAND,FINTRNTOT,FINTXBEXTOT,FINTXBTOT
COMMUNITY_BOARD,,,,,,,,,,
5.0,975475.333333,117187.5,1001887.5,260416.666667,2.226417e+06,983153.75,95712.25,1010741.0,975475.333333,1001887.5


### Bronx - CD 01

In [200]:
# Bronx-01 subset
bx1 = corpad2[(corpad2.BOROUGH =='Bronx') & (corpad2.COMMUNITY_BOARD == 1)]
bx1

,JOB_NUMBER,JOB_TYPE,C_O_ISSUE_DATE,BIN_NUMBER,BOROUGH,NUMBER,STREET,BLOCK,LOT,POSTCODE,...,STORAGE_AREA_GROSS,GARAGE_AREA,OTHER_AREA_GROSS,REUC_DESCRIPTION,EXTRACRDT,PYTAXFLAG,TENTAXFLAG,CBNTAXFLAG,FINTAXFLAG,CURTAXFLAG
360,220407842,NB,2018-08-16,2128617,Bronx,532,E 142 STREET,02268,0018,10454.0,...,0.0,0.0,0.0,NaN,05/17/2020,A,A,A,A,A
361,220407842,NB,2018-08-16,2128617,Bronx,532,E 142 STREET,02268,0018,10454.0,...,0.0,0.0,0.0,NaN,05/24/2021,A,A,A,A,A
362,200483039,NB,2017-11-29,2117110,Bronx,521,EAST 149 STREET,02276,0054,10455.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
363,220516216,NB,2020-05-29,2127605,Bronx,485,EAST 139TH STREET,02284,0034,10454.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364,220471683,NB,2018-03-22,2818714,Bronx,352,WILLIS AVE,02286,0012,10454.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
365,220072463,NB,2016-09-26,2831652,Bronx,356,WILLIS AVENUE,02287,0002,10454.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
366,200813058,NB,2019-05-21,2120053,Bronx,456,WILLIS AVENUE,02290,0005,10455.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
367,220420550,NB,2019-09-26,2805101,Bronx,428,E. 148 STREET,02292,0026,10455.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
368,220124381,NB,2016-12-23,2826369,Bronx,469,EAST 147 STREET,02292,0049,10455.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
369,220111689,NB,2016-12-28,2826368,Bronx,463,EAST 147 STREET,02292,0050,10455.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [201]:
# variables of interest
pd.pivot_table(bx1, values = ['FINMKTLAND','FINMKTTOT','FINACTLAND','FINACTTOT','FINACTEXTOT','FINTRNLAND','FINTRNTOT','FINTRNEXTOT','FINTXBTOT','FINTXBEXTOT'], index = ['COMMUNITY_BOARD'])

,FINACTEXTOT,FINACTLAND,FINACTTOT,FINMKTLAND,FINMKTTOT,FINTRNEXTOT,FINTRNLAND,FINTRNTOT,FINTXBEXTOT,FINTXBTOT
COMMUNITY_BOARD,,,,,,,,,,
1.0,4647793.1,439650.0,4718047.5,977000.0,10484550.0,4710895.0,381509.35,4781393.35,4646326.4,4716566.0


### Bronx - CD 06

In [191]:
# Bronx-06 subset
bx6 = corpad2[(corpad2.BOROUGH =='Bronx') & (corpad2.COMMUNITY_BOARD == 6)]
bx6

,JOB_NUMBER,JOB_TYPE,C_O_ISSUE_DATE,BIN_NUMBER,BOROUGH,NUMBER,STREET,BLOCK,LOT,POSTCODE,...,STORAGE_AREA_GROSS,GARAGE_AREA,OTHER_AREA_GROSS,REUC_DESCRIPTION,EXTRACRDT,PYTAXFLAG,TENTAXFLAG,CBNTAXFLAG,FINTAXFLAG,CURTAXFLAG
548,220152420,NB,2019-12-09,2128726,Bronx,1880,BATHGATE AVENUE,02924,0007,10457.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
552,220520498,NB,2018-10-22,2129012,Bronx,712,EAST 175 STREET,02948,0009,10457.0,...,0.0,0.0,0.0,NaN,05/17/2020,A,A,A,A,A
553,220520498,NB,2018-10-22,2129012,Bronx,712,EAST 175 STREET,02948,0009,10457.0,...,0.0,0.0,0.0,NaN,05/24/2021,A,A,A,A,A
554,200909712,NB,2017-11-13,2124748,Bronx,707,EAST 175 STREET,02949,0037,10457.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
555,200909703,NB,2017-11-22,2806599,Bronx,705,EAST 175 STREET,02949,0038,10457.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
556,200976890,NB,2019-12-10,2115050,Bronx,818,ELSMERE PLACE,02955,0130,10460.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
557,200899876,NB,2017-07-05,2115905,Bronx,862,EAST 175 STREET,02957,0031,10460.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
577,220542401,NB,2018-06-21,2120210,Bronx,1880,BOSTON ROAD,03004,0020,10460.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
578,201197702,NB,2020-01-24,2010892,Bronx,1930,VYSE AVENUE,03005,0010,10460.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
584,220152019,NB,2018-11-13,2129118,Bronx,1903,WEST FARMS ROAD,03016,0011,10460.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [192]:
# variables of interest
pd.pivot_table(bx6, values = ['FINMKTLAND','FINMKTTOT','FINACTLAND','FINACTTOT','FINACTEXTOT','FINTRNLAND','FINTRNTOT','FINTRNEXTOT','FINTXBTOT','FINTXBEXTOT'], index = ['COMMUNITY_BOARD'])

,FINACTEXTOT,FINACTLAND,FINACTTOT,FINMKTLAND,FINMKTTOT,FINTRNEXTOT,FINTRNLAND,FINTRNTOT,FINTXBEXTOT,FINTXBTOT
COMMUNITY_BOARD,,,,,,,,,,
6.0,3468036.6,274320.0,3554460.0,609600.0,7898800.0,3215766.6,266115.1,3302190.0,3468036.6,3554460.0
